In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

i = 0
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        i+=1
#         print(os.path.join(dirname, filename))
print("Num files: ", i)

Num files:  22176


In [2]:
!git clone --branch feat/Task2_build_model --single-branch https://github.com/thanhdangg/PBL6

Cloning into 'PBL6'...
remote: Enumerating objects: 154, done.
remote: Counting objects: 100% (154/154), done.
remote: Compressing objects: 100% (98/98), done.
remote: Total 154 (delta 77), reused 112 (delta 42), pack-reused 0 (from 0)
Receiving objects: 100% (154/154), 536.42 KiB | 3.19 MiB/s, done.
Resolving deltas: 100% (77/77), done.


In [3]:
%cd /kaggle/working/PBL6/Task2_Lesion_Attribute_Detection

/kaggle/working/PBL6/Task2_Lesion_Attribute_Detection


In [4]:
!python image_preprocess.py --input_dir /kaggle/input/isic-2018-task-2-lesion-attribute-detection/ISIC2018_Task1-2_Training_Input/ISIC2018_Task1-2_Training_Input  --output_dir /kaggle/working/PBL6/output_train/image  --ground_truth_dir /kaggle/input/isic-2018-task-2-lesion-attribute-detection/ISIC2018_Task2_Training_GroundTruth_v3/ISIC2018_Task2_Training_GroundTruth_v3  --output_mask_dir /kaggle/working/PBL6/output_train/mask   --input_val_dir /kaggle/input/isic-2018-task-2-lesion-attribute-detection/ISIC2018_Task1-2_Validation_Input/ISIC2018_Task1-2_Validation_Input   --ground_truth_val_dir /kaggle/input/isic-2018-task-2-lesion-attribute-detection/ISIC2018_Task2_Validation_GroundTruth/ISIC2018_Task2_Validation_GroundTruth   --output_val_dir /kaggle/working/PBL6/output_val/image   --output_mask_val_dir /kaggle/working/PBL6/output_val/mask

Processing for train
Processing ISIC_0012706.jpg
Processing ISIC_0010192.jpg
Processing ISIC_0010017.jpg
Processing ISIC_0013000.jpg
Processing ISIC_0013166.jpg
Processing ISIC_0003308.jpg
Processing ISIC_0012332.jpg
Processing ISIC_0014291.jpg
Processing ISIC_0014729.jpg
Processing ISIC_0000012.jpg
Processing ISIC_0012855.jpg
Processing ISIC_0012789.jpg
Processing ISIC_0014190.jpg
Processing ISIC_0013982.jpg
Processing ISIC_0009083.jpg
Processing ISIC_0010237.jpg
Processing ISIC_0014898.jpg
Processing ISIC_0010332.jpg
Processing ISIC_0016066.jpg
Processing ISIC_0010100.jpg
Processing ISIC_0015034.jpg
Processing ISIC_0012679.jpg
Processing ISIC_0013670.jpg
Processing ISIC_0015129.jpg
Processing ISIC_0014393.jpg
Processing ISIC_0012510.jpg
Processing ISIC_0013962.jpg
Processing ISIC_0010263.jpg
Processing ISIC_0013487.jpg
Processing ISIC_0000537.jpg
Processing ISIC_0014800.jpg
Processing ISIC_0012382.jpg
Processing ISIC_0012806.jpg
Processing ISIC_0012086.jpg
Processing ISIC_0015158.jpg

In [6]:
!python train.py --input_folder /kaggle/working/PBL6/output_train/image  --ground_truth_dir /kaggle/working/PBL6/output_train/mask  --val_input_folder /kaggle/working/PBL6/output_val/image  --val_ground_truth_dir /kaggle/working/PBL6/output_val/mask  --output_model_h5 /kaggle/working/models/multi_task_unet.h5  --output_model_keras /kaggle/working/models/multi_task_unet.keras --plot_output_dir /kaggle/working/plots --epochs 20  --batch_size 32  

Loading dataset...
Creating DataLoader...
Loading dataloader...
Creating Model...
Model:  UNet(
  (encoder1): EncoderBlock(
    (conv_block): ConvBlock(
      (conv): Sequential(
        (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (1): ReLU(inplace=True)
        (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (3): ReLU(inplace=True)
      )
    )
    (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (encoder2): EncoderBlock(
    (conv_block): ConvBlock(
      (conv): Sequential(
        (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (1): ReLU(inplace=True)
        (2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (3): ReLU(inplace=True)
      )
    )
    (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (encoder3): EncoderBlock(
    (conv_block): ConvBlock(
      (conv): Sequential(

In [8]:
import torch
import torch.nn as nn
import torch.nn.functional as F
class ConvBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(ConvBlock, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1),
            nn.ReLU(inplace=True)
        )
        
    def forward(self, x):
        return self.conv(x)

class EncoderBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(EncoderBlock, self).__init__()
        self.conv_block = ConvBlock(in_channels, out_channels)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
    
    def forward(self, x):
        conv_out = self.conv_block(x)
        pool_out = self.pool(conv_out)
        return conv_out, pool_out


class Bottleneck(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(Bottleneck, self).__init__()
        self.conv_block = ConvBlock(in_channels, out_channels)
    
    def forward(self, x):
        return self.conv_block(x)


class DecoderBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(DecoderBlock, self).__init__()
        self.upconv = nn.ConvTranspose2d(in_channels, out_channels, kernel_size=2, stride=2)
        self.conv_block = ConvBlock(in_channels, out_channels)  # Concatenated output will have in_channels * 2
    
    def forward(self, x, enc_output):
        x = self.upconv(x)
        # Concatenate along the channel axis (dim=1)
        x = torch.cat((x, enc_output), dim=1)
        return self.conv_block(x)


class UNet(nn.Module):
    def __init__(self, in_channels=3, out_channels=5):
        super(UNet, self).__init__()

        # Encoder blocks
        self.encoder1 = EncoderBlock(in_channels, 64)
        self.encoder2 = EncoderBlock(64, 128)
        self.encoder3 = EncoderBlock(128, 256)
        self.encoder4 = EncoderBlock(256, 512)

        # Bottleneck
        self.bottleneck = Bottleneck(512, 1024)

        # Decoder blocks
        self.decoder4 = DecoderBlock(1024, 512)
        self.decoder3 = DecoderBlock(512, 256)
        self.decoder2 = DecoderBlock(256, 128)
        self.decoder1 = DecoderBlock(128, 64)

        # Final output layer
        self.final_conv = nn.Conv2d(64, out_channels, kernel_size=1)
        
    def forward(self, x):
        # Encoder path
        enc1, pool1 = self.encoder1(x)
        enc2, pool2 = self.encoder2(pool1)
        enc3, pool3 = self.encoder3(pool2)
        enc4, pool4 = self.encoder4(pool3)

        # Bottleneck
        bottleneck = self.bottleneck(pool4)

        # Decoder path
        dec4 = self.decoder4(bottleneck, enc4)
        dec3 = self.decoder3(dec4, enc3)
        dec2 = self.decoder2(dec3, enc2)
        dec1 = self.decoder1(dec2, enc1)
        
        out = self.final_conv(dec1)
        return out
    
# def create_unet_model(in_channels=3, out_channels=1, init_features=32):
#     model = UNet(in_channels, out_channels, init_features)
#     return model


# if __name__ == "__main__":
#     # Create the U-Net model
#     model = UNet(in_channels=3, out_channels=1)

#     # Test with a random input tensor
#     input_image = torch.randn((1, 3, 256, 256))  # Example input (batch_size=1, channels=3, height=256, width=256)
#     output_mask = model(input_image)

#     print(f"Output shape: {output_mask.shape}")  # Expected output shape: [1, 1, 256, 256]


In [20]:
import argparse
import os
import torch
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
from PIL import Image
from model import UNet
from loss import HybridLoss
import numpy as np
import matplotlib.pyplot as plt

class TestSkinLesionDataset(Dataset):
    def __init__(self, image_dir, mask_dir, target_size, attributes):
        self.image_dir = image_dir
        self.mask_dir = mask_dir
        self.target_size = target_size
        self.attributes = attributes
        self.image_filenames = [f for f in os.listdir(image_dir) if f.endswith('.jpg') or f.endswith('.png')]
        self.transform = transforms.Compose([
            transforms.Resize(target_size),
            transforms.ToTensor()
        ])

    def __len__(self):
        return len(self.image_filenames)

    def __getitem__(self, idx):
        image_filename = self.image_filenames[idx]
        image_path = os.path.join(self.image_dir, image_filename)
        image = Image.open(image_path).convert('RGB')
        image = self.transform(image)

        masks = []
        for attribute in self.attributes:
            # mask_filename = image_filename.replace('.jpg', f'_attribute_{attribute}.png').replace('.png', f'_attribute_{attribute}.png')
            mask_filename = image_filename.replace('.jpg', f'_attribute_{attribute}.png')

            # mask_filename = image_filename
            mask_path = os.path.join(self.mask_dir, mask_filename)
            mask = Image.open(mask_path).convert('L')
            mask = self.transform(mask)
            mask = (mask > 0).float()  # Convert mask to binary
            masks.append(mask)

        masks = torch.cat(masks, dim=0)  # Concatenate masks along the channel dimension

        return image, masks, image_filename
    
def is_blank_image(image):
    return np.all(image == image[0, 0])

def is_blank_mask(pred_mask, threshold=0.5):
    """
    Check if the predicted mask is blank.
    A mask is considered blank if all values are below the threshold.
    """
    return torch.all(pred_mask < threshold)


# Function to save the mask, handling blank masks
def save_mask(pred_mask, output_path, threshold=0.5):
    """
    Save the mask as a PNG image.
    If the prediction is a blank mask, output a black image.
    """
    if is_blank_mask(pred_mask, threshold):
        # Create a black image if it is a blank mask
        blank_image = Image.new("L", pred_mask.shape[-2:])  # "L" mode for grayscale image
        blank_image.save(output_path)
        print(f"Blank mask saved as {output_path}")
    else:
        # Convert tensor to image and save
        pred_mask_np = pred_mask.cpu().detach().numpy()
        binary_mask = (pred_mask_np > threshold).astype(np.uint8) * 255  # Convert to 0 or 255
        mask_image = Image.fromarray(binary_mask.squeeze(), mode="L")
        mask_image.save(output_path)
        print(f"Mask saved as {output_path}")
        
def evaluate_model(model, dataloader, device, output_dir):
    model = model.to(device)
    model.eval()  # Set model to evaluation mode
    criterion = HybridLoss()
    running_loss = 0.0

    os.makedirs(output_dir, exist_ok=True)
    
    attributes = ["pigment_network", "negative_network", "streaks", "milia_like_cyst", "globules"]


    with torch.no_grad():  # Disable gradient computation
        for images, masks, filenames in dataloader:
            images = images.to(device)
            masks = masks.to(device)

            # Forward pass
            outputs = model(images)
            loss = criterion(outputs, masks)

            # Update running loss
            running_loss += loss.item() * images.size(0)

            # Save predicted masks
            for i in range(outputs.size(0)):
                for j, attribute in enumerate(attributes):
                    # pred_mask = outputs[i, j].cpu().numpy().squeeze()
                    # pred_mask = (pred_mask > 0.5).astype(np.uint8) * 255  # Convert to binary mask
                    # pred_image = Image.fromarray(pred_mask)
                    # pred_image.save(os.path.join(output_dir, f"{filenames[i]}_{attribute}.png"))
                    
                    pred_mask = outputs[i, j]
                    output_path = os.path.join(output_dir, f"{filenames[i]}_{attribute}.png")
                    save_mask(pred_mask, output_path)  

    avg_loss = running_loss / len(dataloader.dataset)
    print(f"Test Loss: {avg_loss:.4f}")

    return avg_loss





In [21]:
input_folder = '/kaggle/input/isic-2018-task-2-lesion-attribute-detection/ISIC2018_Task1-2_Validation_Input/ISIC2018_Task1-2_Validation_Input'
mask_folder =  '/kaggle/input/isic-2018-task-2-lesion-attribute-detection/ISIC2018_Task2_Validation_GroundTruth/ISIC2018_Task2_Validation_GroundTruth'
size = (256, 256)
batch_size = 32
model_path = '/kaggle/working/models/multi_task_unet.h5'
output_dir =  '/kaggle/working/predictions'

attributes = ["pigment_network", "negative_network", "streaks", "milia_like_cyst", "globules"]
dataset = TestSkinLesionDataset(image_dir=input_folder, mask_dir=mask_folder, target_size=size, attributes=attributes)
print('Loading dataset...', dataset)
print("Creating DataLoader...")
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False)
print("Loading dataloader...", dataloader)

model = UNet()
model.load_state_dict(torch.load(model_path))
print("Model loaded from", model_path)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Testing on device: {device}")

avg_loss = evaluate_model(model, dataloader, device, output_dir)
print(f"Average Test Loss: {avg_loss:.4f}")

Loading dataset... <__main__.TestSkinLesionDataset object at 0x7a4f987f6e90>
Creating DataLoader...
Loading dataloader... <torch.utils.data.dataloader.DataLoader object at 0x7a4ee15aa290>


/tmp/ipykernel_36/3377403342.py:16: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path))


Model loaded from /kaggle/working/models/multi_task_unet.h5
Testing on device: cuda
Blank mask saved as /kaggle/working/predictions/ISIC_0017341.jpg_pigment_network.png
Blank mask saved as /kaggle/working/predictions/ISIC_0017341.jpg_negative_network.png
Blank mask saved as /kaggle/working/predictions/ISIC_0017341.jpg_streaks.png
Blank mask saved as /kaggle/working/predictions/ISIC_0017341.jpg_milia_like_cyst.png
Blank mask saved as /kaggle/working/predictions/ISIC_0017341.jpg_globules.png
Blank mask saved as /kaggle/working/predictions/ISIC_0015480.jpg_pigment_network.png
Blank mask saved as /kaggle/working/predictions/ISIC_0015480.jpg_negative_network.png
Blank mask saved as /kaggle/working/predictions/ISIC_0015480.jpg_streaks.png
Blank mask saved as /kaggle/working/predictions/ISIC_0015480.jpg_milia_like_cyst.png
Blank mask saved as /kaggle/working/predictions/ISIC_0015480.jpg_globules.png
Mask saved as /kaggle/working/predictions/ISIC_0015518.jpg_pigment_network.png
Blank mask save

In [7]:
!python test.py --input_folder /kaggle/input/isic-2018-task-2-lesion-attribute-detection/ISIC2018_Task1-2_Test_Input/ISIC2018_Task1-2_Test_Input --mask_folder /kaggle/input/isic-2018-task-2-lesion-attribute-detection/ISIC2018_Task2_Test_GroundTruth/ISIC2018_Task2_Test_GroundTruth --batch_size 32 --model_path /kaggle/working/models/multi_task_unet.h5 --output_dir /kaggle/working/predictions 

Loading dataset... <__main__.TestSkinLesionDataset object at 0x7d716d43ace0>
Creating DataLoader...
Loading dataloader... <torch.utils.data.dataloader.DataLoader object at 0x7d716d43ada0>
/kaggle/working/PBL6/Task2_Lesion_Attribute_Detection/test.py:140: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you do

In [ ]:
import torch
from model import UNet  # Đảm bảo import đúng mô hình UNet mà bạn dùng

# Định nghĩa mô hình
model = UNet()

# Tải trọng số mô hình đã lưu
model.load_state_dict(torch.load("/kaggle/working/models/multi_task_unet.h5"))

# Chuyển mô hình sang GPU nếu có
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)


In [ ]:
from torch.utils.data import DataLoader
from datasets import SkinLesionDataset  # Lớp Dataset bạn đã định nghĩa
from loss import HybridLoss  # Loss function bạn sử dụng

# Thiết lập dataset và dataloader
train_dataset = SkinLesionDataset(image_dir="/kaggle/input/isic-2018-task-2-lesion-attribute-detection/ISIC2018_Task1-2_Training_Input/ISIC2018_Task1-2_Training_Input",
                                  ground_truth_dir="/kaggle/input/isic-2018-task-2-lesion-attribute-detection/ISIC2018_Task2_Training_GroundTruth_v3/ISIC2018_Task2_Training_GroundTruth_v3",
                                  target_size=(256, 256))

val_dataset = SkinLesionDataset(image_dir="/kaggle/input/isic-2018-task-2-lesion-attribute-detection/ISIC2018_Task1-2_Validation_Input/ISIC2018_Task1-2_Validation_Input",
                                ground_truth_dir="/kaggle/input/isic-2018-task-2-lesion-attribute-detection/ISIC2018_Task2_Validation_GroundTruth/ISIC2018_Task2_Validation_GroundTruth",
                                target_size=(256, 256))

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

# Loss và optimizer
criterion = HybridLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)


In [ ]:
from tqdm import tqdm

num_epochs = 10  # Số epochs muốn train tiếp
validation_losses = []
training_losses = []
best_val_loss = float('inf')
output_model_path = "/kaggle/working/models/new_multi_task_unet.h5"


for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    progress_bar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}", unit="batch")
    
    for images, masks in progress_bar:
        images = images.to(device)
        masks = masks.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, masks)

        # Backward pass và optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)
        progress_bar.set_postfix(loss=loss.item())

    # Tính loss trung bình của epoch này
    epoch_loss = running_loss / len(train_loader.dataset)
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}")
    training_losses.append(epoch_loss)


    # Evaluate trên validation set nếu cần
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for val_images, val_masks in val_loader:
            val_images = val_images.to(device)
            val_masks = val_masks.to(device)
            val_outputs = model(val_images)
            val_loss += criterion(val_outputs, val_masks).item() * val_images.size(0)

    val_loss /= len(val_loader.dataset)
    print(f"Validation Loss: {val_loss:.4f}")

    validation_losses.append(val_loss)
    if val_loss < best_val_loss:
            best_val_loss = val_loss
            output_dir = os.path.dirname(output_model_path)
            if not os.path.exists(output_dir):
                os.makedirs(output_dir)
            torch.save(model.state_dict(), output_model_path)
            print(f"Best model saved to {output_model_path}")
    